<a href="https://colab.research.google.com/github/nrocka/Anime-SD-Prompt-to-Danbooru/blob/main/Anime_SD_Prompt_to_Danbooru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🟥🟥🟥 **Preperation** 🟥🟥🟥

In [ ]:
# Import Cell
# @title 0️⃣ Import Dependencies
# @markdown This cell contains all necessary imports.
import re
import requests
import pandas as pd
from IPython.display import display, HTML, Javascript

In [ ]:
# Global Parameters Cell
# @title ✨ Global Parameters used throughout the notebook
# @markdown This parameter defines the maximum number of characters allowed before inserting a newline, making the output more readable on different window sizes. (Will not affect "Copy to Clipboard")
newline_threshold = 240 # @param {type:"integer"}

# @markdown This parameter decides whether a newline should cut off only after completing a tag or directly after reaching the threshold. (Will not affect "Copy to Clipboard")
only_cut_after_complete_tags = True # @param {type:"boolean"}

## 🟩🟩🟩 **Prompt Editor** 🟩🟩🟩

In [81]:
# First Cell: Clean Up Prompt
# @title 1️⃣ Clean Up Prompt
# @markdown This cell removes unwanted text inside chevrons and cleans up the prompt.

input_prompt = "" # @param {type:"string"}

def clean_up_prompt(input_prompt):
    # 1. Remove chevrons (< and >) and all text inside them
    cleaned_prompt = re.sub(r'<.*?>', '', input_prompt)

    # 2. Remove weights from tags in brackets, e.g., (solo, 1girl:1.3) -> (solo, 1girl)
    cleaned_prompt = re.sub(r'\(([^:]*):[^\)]*\)', r'(\1)', cleaned_prompt)

    # 3. Remove all brackets ( ), but exclude brackets with slashes in front.
    cleaned_prompt = re.sub(r'(?<!/)\(', '', cleaned_prompt)
    cleaned_prompt = re.sub(r'(?<!/)\)', '', cleaned_prompt)

    # 4. Remove any double spaces between tags
    cleaned_prompt = re.sub(r'\s{2,}', ' ', cleaned_prompt)


    return cleaned_prompt

cleaned_prompt = clean_up_prompt(input_prompt)
print("Cleaned Prompt:")

if only_cut_after_complete_tags:
    output_lines = []
    current_line = ""
for tag in cleaned_prompt.split(', '):
    if len(current_line) + len(tag) + 2 <= newline_threshold:
        current_line += (', ' if current_line else '') + tag
    else:
        current_line = current_line.rstrip(', ') + ','  # Make sure last tag in line ends with a comma
        output_lines.append(current_line)
        current_line = tag
if current_line:
    current_line = current_line.rstrip(', ')  # Remove any trailing commas after processing all lines
    output_lines.append(current_line)
    cleaned_output = '\n'.join(output_lines)
else:
    cleaned_output = '\n'.join([cleaned_prompt[i:i + newline_threshold] for i in range(0, len(cleaned_prompt), newline_threshold)])

print(cleaned_output)

display(HTML(f"""
    <button onclick="navigator.clipboard.writeText('{cleaned_prompt}')">Copy to Clipboard</button>
"""))

Cleaned Prompt:
score_9, score_8_up, score_7_up, re :zero, re: zero, re:zero, source_anime, asuna /(sao/), 1girl, solo, ushio kofune, long hair, blue eyes, blonde hair, very long hair, straight hair, forehead, jewelry, collarbone, swimsuit, necklace,
one-piece swimsuit, school swimsuit, blue one-piece swimsuit, competition school swimsuit, shell necklace, one-piece swimsuit pull, pulled by self, undressing, bare shoulders, nipples, smirk, smile, medium breasts, breasts out, ocean, water,
blue sky, cloudy sky






---




In [60]:
# Second Cell: Remove Quality, Source and Score Tags
# @title 2️⃣ Remove Quality, Source, and Score Tags
# @markdown This cell removes score_, source_, quality tags, and date tags.

def remove_quality_source_tags(cleaned_prompt):
    # 1. Remove all score_ and source_ prompts
    cleaned_prompt = re.sub(r'\b(score_|source_)[^,\s]*[\s,]*', '', cleaned_prompt)

    # 2. Remove all quality tags used by danbooru
    quality_tags = ["masterpiece", "best quality"]
    for tag in quality_tags:
        cleaned_prompt = cleaned_prompt.replace(tag, '')

    # 3. Remove all date tags like Year Range
    date_tags = ["old", "early", "mid", "recent", "newest"]
    for tag in date_tags:
        cleaned_prompt = cleaned_prompt.replace(tag, '')

    # Clean up extra spaces and commas left behind
    cleaned_prompt = re.sub(r',\s*', ', ', cleaned_prompt).strip(', ')  # Remove trailing commas
    cleaned_prompt = re.sub(r'\s+', ' ', cleaned_prompt).strip()  # Remove excess spaces

    return cleaned_prompt

cleaned_prompt = remove_quality_source_tags(cleaned_prompt)
if only_cut_after_complete_tags:
    output_lines = []
    current_line = ""
for tag in cleaned_prompt.split(', '):
    if len(current_line) + len(tag) + 2 <= newline_threshold:
        current_line += (', ' if current_line else '') + tag
    else:
        current_line = current_line.rstrip(', ') + ','  # Make sure last tag in line ends with a comma
        output_lines.append(current_line)
        current_line = tag
if current_line:
    current_line = current_line.rstrip(', ')  # Remove any trailing commas after processing all lines
    output_lines.append(current_line)
    cleaned_output = '\n'.join(output_lines)
else:
    cleaned_output = '\n'.join([cleaned_prompt[i:i + newline_threshold] for i in range(0, len(cleaned_prompt), newline_threshold)])

print(cleaned_output)

display(HTML(f"""
    <button onclick="navigator.clipboard.writeText('{cleaned_prompt}')">Copy to Clipboard</button>
"""))



---



In [61]:
# Third Cell: Format to Danbooru Input
# @title 3️⃣ Format to Danbooru Input
# @markdown This cell replaces spaces in tags with underscores and removes commas.

def format_to_danbooru_input(cleaned_prompt):
    # 1. Replace the spaces in every individual tag with underscores (_)
    tags = cleaned_prompt.split(', ')
    tags = [tag.strip().replace(' ', '_') for tag in tags]

    # 2. Remove every comma
    formatted_prompt = ' '.join(tags)

    return formatted_prompt

formatted_prompt = format_to_danbooru_input(cleaned_prompt)
print("Formatted Prompt for Danbooru:")

if only_cut_after_complete_tags:
    output_lines = []
    current_line = ""
    for tag in formatted_prompt.split(' '):
        if len(current_line) + len(tag) + 1 <= newline_threshold:  # Use 1 instead of 2 since there's no comma
            current_line += (' ' if current_line else '') + tag
        else:
            output_lines.append(current_line)
            current_line = tag
    if current_line:
        output_lines.append(current_line)
    formatted_output = '\n'.join(output_lines)
else:
    formatted_output = '\n'.join([formatted_prompt[i:i + newline_threshold] for i in range(0, len(formatted_prompt), newline_threshold)])

print(formatted_output)

display(HTML(f"""
    <button onclick="navigator.clipboard.writeText('{formatted_prompt}')">Copy to Clipboard</button>
"""))

Formatted Prompt for Danbooru:





---



In [62]:
# Fourth Cell: List Unknown Tags
# @title 4️⃣ List unsupported and unknown Tags
# @markdown This cell lists unsupported or deprecated tags based on a CSV file containing selected tags. Chose the output:

output_option = "A - Only return unsupported and unknown tags" # @param ["A - Only return unsupported and unknown tags", "B - Return set of tags without unsupported and unknown tags", "C - Both A and B"]

def check_tags_against_csv(formatted_prompt, option):
    # Load selected tags from CSV
    # @markdown Chose a custom CSV to compare your tags to. A Tag will be listed as unsupported if it can't be found inside this CSV.
    # Default = https://huggingface.co/SmilingWolf/wd-vit-large-tagger-v3/raw/main/selected_tags.csv
    csv_url = "https://huggingface.co/SmilingWolf/wd-vit-large-tagger-v3/raw/main/selected_tags.csv" # @param {type:"string"}
    url = csv_url
    selected_tags = pd.read_csv(url)['name'].tolist()

    # Split tags from formatted prompt
    tags = formatted_prompt.split(' ')

    unsupported_tags = []
    supported_tags = []

    # Check each tag against the list of selected tags
    for tag in tags:
        if tag not in selected_tags:
            unsupported_tags.append(tag)
        else:
            supported_tags.append(tag)

    if option == "A" or option == "C":
        # Only return unsupported and unknown tags
        if unsupported_tags:
            unsupported_output = "Unsupported Tags: " + ', '.join(unsupported_tags)

            if only_cut_after_complete_tags:
                output_lines = []
                current_line = ""
                for tag in unsupported_tags:
                    if len(current_line) + len(tag) + 2 <= newline_threshold:
                        current_line += (', ' if current_line else '') + tag
                    else:
                        output_lines.append(current_line)
                        current_line = tag
                if current_line:
                    output_lines.append(current_line)
                unsupported_output = '\n'.join(output_lines)
            else:
                unsupported_output = '\n'.join([unsupported_output[i:i + newline_threshold] for i in range(0, len(unsupported_output), newline_threshold)])

            print("Unsupported Tags:")
            print(unsupported_output)
            display(HTML(f"""
                <button onclick="navigator.clipboard.writeText('{', '.join(unsupported_tags)}')">Copy to Clipboard</button>
            """))
        else:
            print("No unsupported tags found.")

    if option == "B" or option == "C":
        # Return set of tags without unsupported and unknown tags
        if supported_tags:
            supported_output = "Supported Tags: " + ', '.join(supported_tags)

            if only_cut_after_complete_tags:
                output_lines = []
                current_line = ""
                for tag in supported_tags:
                    if len(current_line) + len(tag) + 2 <= newline_threshold:
                        current_line += (', ' if current_line else '') + tag
                    else:
                        output_lines.append(current_line)
                        current_line = tag
                if current_line:
                    output_lines.append(current_line)
                supported_output = '\n'.join(output_lines)
            else:
                supported_output = '\n'.join([supported_output[i:i + newline_threshold] for i in range(0, len(supported_output), newline_threshold)])

            print("Supported Tags:")
            print(supported_output)
            display(HTML(f"""
                <button onclick="navigator.clipboard.writeText('{', '.join(supported_tags)}')">Copy to Clipboard</button>
            """))

# Use the formatted prompt from the previous cell
check_tags_against_csv(formatted_prompt, output_option)




---



In [63]:
# Clear Inputs/Outputs Cell
# @title ✨ Clear All Inputs and Outputs
# @markdown This cell resets all inputs and outputs to a clean state.

def clear_all():
    global input_prompt, cleaned_prompt, formatted_prompt
    input_prompt = ""
    cleaned_prompt = ""
    formatted_prompt = ""
    print("All inputs and outputs have been cleared.")

clear_all()

All inputs and outputs have been cleared.
